# Chatbot con spacy

In [42]:
# pip install spacy
# python -m spacy download en_core_web_lg
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

nlp = spacy.load('en_core_web_lg')

# Define possible answers
answers = [ "Dungeons & Dragons is a fantasy tabletop role-playing game.",
            "To create a character, you'll choose a race, class, abilities, and background.",
            "Classes are fighters, wizards, rogues, clerics, paladins, rangers, and more.",
            "Ability scores represent a character's innate abilities and strengths. They include Strength, Dexterity, Constitution, Intelligence, Wisdom, and Charisma.",
            "Combat in D&D is turn-based, with each character and creature taking actions in a specific order.",
            "Spells are magical abilities that characters can cast to produce various effects, such as dealing damage, healing allies, or manipulating the environment.",
            "Some famous D&D campaigns include 'Curse of Strahd,' 'Tomb of Annihilation,' 'Storm King's Thunder,' 'Dragon Heist,' and 'Out of the Abyss,' among others.",
            "The Dungeon Master (DM) is the player who acts as the game's storyteller and referee, guiding the players through the adventure, describing scenes, and controlling non-player characters (NPCs).",
            "Sorcerers derive their magical abilities from innate talent, wizards learn magic through study and practice, and warlocks make pacts with otherworldly beings for their magical powers.",
            "Alignment describes a character's moral and ethical outlook. The alignment help define a character's beliefs and motivations."]

# Generate embeddings for each answer
#embeddings = [nlp(answer.lower()).vector for answer in answers]
embeddings = []
for answer in answers:
    embeddings.append(np.array([t.vector for t in nlp(answer.lower())]).mean(0).reshape(1,-1))

knowledge_dataset = []
for i in range(0, len(answers)):
    knowledge_dataset.append((answers[i], embeddings[i]))


def get_userQuestionEmbedding(user_question):
    return np.array([t.vector for t in nlp(user_question.lower())]).mean(0).reshape(1,-1)

def get_most_similar_answer(user_question):
    user_question_embedding = get_userQuestionEmbedding(user_question)
    max_similarity = -1
    most_similar_answer = ""
    for answer, answer_embedding in knowledge_dataset:
        print(cosine_similarity(user_question_embedding, answer_embedding))
        similarity = cosine_similarity(user_question_embedding, answer_embedding)

        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_answer = answer

    return most_similar_answer

## Posibles preguntas

* What is Dungeons & Dragons?
* How do I create a character?
* What are the different classes?
* What are ability scores?
* How does combat work?
* What are spells?
* What are some famous campaigns or adventures?
* What is a Dungeon Master (DM)?
* Can you explain the difference between a sorcerer, wizard, and warlock?
* How alignment work?

In [44]:
user_question = "What is Dungeons & Dragons?"
answer = get_most_similar_answer(user_question)
print(answer)

[[0.58590955]]
[[0.2724832]]
[[0.27344286]]
[[0.29088792]]
[[0.29198083]]
[[0.31451812]]
[[0.18253727]]
[[0.25038144]]
[[0.32359082]]
[[0.34910378]]
Dungeons & Dragons is a fantasy tabletop role-playing game.
